# Import SQLite databse from Google Drive
---

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sqlite3

conn = sqlite3.connect('linkedin_jobs.db')
cur  = conn.cursor()